In [3]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
#import matplotlib.pyplot as plt

#from pmdarima import auto_arima
from joblib import Parallel, delayed
from sklearn.metrics import (precision_score, recall_score,
                           f1_score, roc_auc_score, confusion_matrix)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Fusion

In [4]:
fusion_test = np.load('/home/haoqian/anomaly/features/fusion/fusion_test.npy')
fusion_train = np.load('/home/haoqian/anomaly/features/fusion/fusion_train.npy')
label = np.load('/home/haoqian/anomaly/SMD/SMD_test_label.npy')

In [5]:
# Subsample
def fixed_interval_sampling(data, interval=360):
    """
    Fixed-interval sampling with the same start point for all features.
    Interval: 360 => 4 samples/day (if data is 10-min frequency)
    """
    n_samples, n_features = data.shape
    max_samples = n_samples // interval
    sampled_data = np.zeros((max_samples, n_features))
    start = np.random.randint(0, interval)
    indices = np.arange(start, start + max_samples * interval, interval)
    indices = np.clip(indices, 0, n_samples - 1)
    sampled_data = data[indices, :]
    return sampled_data, start

In [6]:
interval = 360
fusion_all = np.concatenate([fusion_train, fusion_test], axis=0)
fusion_all_samples, start = fixed_interval_sampling(fusion_all, interval)
n_train_samples = (fusion_train.shape[0] - start - 1) // interval + 1
#train
sampled_train = fusion_all_samples[:n_train_samples]
#test
sampled_test = fusion_all_samples[n_train_samples:]
#label
start_label=start+n_train_samples*interval-fusion_train.shape[0]
indices = np.arange(start_label, len(label), interval)
indices = np.clip(indices, 0, len(label) - 1)
sampled_labels = label[indices]
if len(sampled_labels) > sampled_test.shape[0]:
    sampled_labels = sampled_labels[:sampled_test.shape[0]]
elif len(sampled_labels) < sampled_test.shape[0]:
    sampled_labels = np.pad(sampled_labels, (0, sampled_test.shape[0] - len(sampled_labels)), mode='edge')

## ARIMA

In [7]:
def grid_search_arima(train_series, p_range=(1, 4), q_range=(1, 3), d=1):
    best_aic = np.inf
    best_order = (1, 1, 1)
    best_model = ARIMA(train_series, order=(1,1,1)).fit()

    for p in range(*p_range):
        for q in range(*q_range):
            try:
                model = ARIMA(train_series, order=(p, d, q))
                results = model.fit()
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = (p, d, q)
                    best_model = results
            except:
                continue
    return best_model, best_order

def train_arima_for_feature(train_series, test_series):
    model, order = grid_search_arima(train_series, d=1)  # fix d=1
    forecast = model.forecast(steps=len(test_series))
    errors = np.abs(test_series - forecast)
    return {
        'model': model,
        'order': order,
        'forecast': forecast,
        'errors': errors
    }
print("Training ARIMA models with manual grid search (d=1)...")
arima_results = Parallel(n_jobs=4)(
    delayed(train_arima_for_feature)(sampled_train[:, i], sampled_test[:, i])
    for i in range(sampled_train.shape[1])
)

Training ARIMA models with manual grid search (d=1)...


/home/haoqian/anaconda3/envs/golden/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/haoqian/anaconda3/envs/golden/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/haoqian/anaconda3/envs/golden/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/haoqian/anaconda3/envs/golden/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible startin

In [8]:
def evaluate_anomalies(results, test_data, y_true):
    all_errors = np.array([res['errors'] for res in results]).T
    avg_errors = np.mean(all_errors, axis=1)
    threshold = np.quantile(avg_errors, 0.9)

    pred_labels = (avg_errors > threshold).astype(int)
    return {
        'precision': precision_score(y_true, pred_labels, zero_division=0),
        'recall': recall_score(y_true, pred_labels),
        'f1': f1_score(y_true, pred_labels),
        'roc_auc': roc_auc_score(y_true, avg_errors),
        'confusion_matrix': confusion_matrix(y_true, pred_labels),
        'threshold': threshold,
        'errors': avg_errors,
        'true_labels': y_true
    }

print("Evaluating anomalies...")
eval_results = evaluate_anomalies(arima_results, sampled_test, sampled_labels)
print("\n=== Evaluation Results ===")
print(f"Precision: {eval_results['precision']:.4f}")
print(f"Recall:    {eval_results['recall']:.4f}")
print(f"F1 Score:  {eval_results['f1']:.4f}")
print(f"ROC-AUC:   {eval_results['roc_auc']:.4f}")

Evaluating anomalies...

=== Evaluation Results ===
Precision: 0.1117
Recall:    0.2750
F1 Score:  0.1588
ROC-AUC:   0.7153


## VAR

In [88]:
def grid_search_var(train_data, max_lag=5):
    """
    Grid search for optimal VAR lag order using AIC
    Args:
        train_data: Multivariate time series data (n_samples, n_features)
        max_lag: Maximum lag order to consider
    Returns:
        best_model: Fitted VAR model with optimal lag
        best_lag: Selected lag order
    """
    best_aic = np.inf
    best_lag = 1
    model = VAR(train_data)
    best_model = model.fit(best_lag)

    for lag in range(1, max_lag+1):
        try:
            model = VAR(train_data)
            results = model.fit(lag)
            if results.aic < best_aic:
                best_aic = results.aic
                best_model = results
                best_lag = lag
        except:
            continue

    return best_model, best_lag

def train_var_for_features(train_data, test_data):
    """
    Train VAR model and make forecasts
    Args:
        train_data: Training data (n_samples, n_features)
        test_data: Test data (n_samples, n_features)
    Returns:
        Dictionary containing model and evaluation metrics
    """
    # Standardize data
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_data)
    test_scaled = scaler.transform(test_data)

    # Fit VAR model
    model, lag = grid_search_var(train_scaled)

    # Make forecasts
    forecast_steps = len(test_data)
    forecast = model.forecast(train_scaled[-lag:], steps=forecast_steps)
    forecast = scaler.inverse_transform(forecast)  # Inverse scaling

    # Calculate errors
    errors = np.abs(test_data - forecast)

    return {
        'model': model,
        'lag': lag,
        'forecast': forecast,
        'errors': errors,
        'mse': mean_squared_error(test_data, forecast)
    }
print("Training VAR models with lag order selection...")
var_result = train_var_for_features(sampled_train, sampled_test)

Training VAR models with lag order selection...


In [91]:
def evaluate_var_anomalies(var_result, test_data, y_true):
    avg_errors = np.mean(var_result['errors'], axis=1)
    threshold = np.quantile(avg_errors, 0.9)
    pred_labels = (avg_errors > threshold).astype(int)
    return {
        'precision': precision_score(y_true, pred_labels, zero_division=0),
        'recall': recall_score(y_true, pred_labels),
        'f1': f1_score(y_true, pred_labels),
        'roc_auc': roc_auc_score(y_true, avg_errors),
        'confusion_matrix': confusion_matrix(y_true, pred_labels),
        'threshold': threshold,
        'errors': avg_errors,
        'true_labels': y_true
    }
var_eval = evaluate_var_anomalies(
    var_result=var_result,
    test_data=sampled_test,
    y_true=sampled_labels
)
print("\n=== Evaluation Results ===")
print(f"Precision: {var_eval['precision']:.4f}")
print(f"Recall:    {var_eval['recall']:.4f}")
print(f"F1 Score:  {var_eval['f1']:.4f}")
print(f"ROC-AUC:   {var_eval['roc_auc']:.4f}")


=== Evaluation Results ===
Precision: 0.1320
Recall:    0.3467
F1 Score:  0.1912
ROC-AUC:   0.7157


# Original

In [92]:
SMD_test = np.load('/content/drive/MyDrive/fusion/SMD_test.npy')
SMD_train = np.load('/content/drive/MyDrive/fusion/SMD_train.npy')
label = np.load('/content/drive/MyDrive/fusion/SMD_test_label.npy')

In [94]:
interval = 360
SMD_all = np.concatenate([SMD_train, SMD_test], axis=0)
SMD_all_samples, start = fixed_interval_sampling(SMD_all, interval)
n_train_samples = (SMD_train.shape[0] - start - 1) // interval + 1
#train
sampled_train = SMD_all_samples[:n_train_samples]
#test
sampled_test = SMD_all_samples[n_train_samples:]
#label
start_label=start+n_train_samples*interval-SMD_train.shape[0]
indices = np.arange(start_label, len(label), interval)
indices = np.clip(indices, 0, len(label) - 1)
sampled_labels = label[indices]
if len(sampled_labels) > sampled_test.shape[0]:
    sampled_labels = sampled_labels[:sampled_test.shape[0]]
elif len(sampled_labels) < sampled_test.shape[0]:
    sampled_labels = np.pad(sampled_labels, (0, sampled_test.shape[0] - len(sampled_labels)), mode='edge')

## ARIMA

In [95]:
print("Training ARIMA models with manual grid search (d=1)...")
arima_results = Parallel(n_jobs=4)(
    delayed(train_arima_for_feature)(sampled_train[:, i], sampled_test[:, i])
    for i in range(sampled_train.shape[1])
)

Training ARIMA models with manual grid search (d=1)...


In [96]:
print("Evaluating anomalies...")
eval_results = evaluate_anomalies(arima_results, sampled_test, sampled_labels)
print("\n=== Original ARIMA Evaluation Results ===")
print(f"Precision: {eval_results['precision']:.4f}")
print(f"Recall:    {eval_results['recall']:.4f}")
print(f"F1 Score:  {eval_results['f1']:.4f}")
print(f"ROC-AUC:   {eval_results['roc_auc']:.4f}")

Evaluating anomalies...

=== Original ARIMA Evaluation Results ===
Precision: 0.0964
Recall:    0.2184
F1 Score:  0.1338
ROC-AUC:   0.6185


## VAR

In [97]:
print("Training VAR models with lag order selection...")
var_result = train_var_for_features(sampled_train, sampled_test)

Training VAR models with lag order selection...


In [98]:
var_eval = evaluate_var_anomalies(
    var_result=var_result,
    test_data=sampled_test,
    y_true=sampled_labels
)
print("\n=== ORIGINAL VAR Evaluation Results ===")
print(f"Precision: {var_eval['precision']:.4f}")
print(f"Recall:    {var_eval['recall']:.4f}")
print(f"F1 Score:  {var_eval['f1']:.4f}")
print(f"ROC-AUC:   {var_eval['roc_auc']:.4f}")


=== ORIGINAL VAR Evaluation Results ===
Precision: 0.1066
Recall:    0.2414
F1 Score:  0.1479
ROC-AUC:   0.6053
